In [2]:
import requests
import json


def getPlaceId(address: str, api_key: str) -> str:
    endpoint_url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json"
    params = {"input": address, "inputtype": "textquery", "key": api_key}

    res = requests.get(endpoint_url, params=params)
    result = json.loads(res.content)

    if result["status"] != "OK":
        raise Exception(result["status"])
    return result["candidates"][0]["place_id"]


def geocode(placeid: str, api_key: str) -> (float, float):
    endpoint_url = "https://maps.googleapis.com/maps/api/place/details/json"
    params = {"place_id": placeid, "key": api_key}

    res = requests.get(endpoint_url, params=params)
    result = json.loads(res.content)

    if result["status"] != "OK":
        raise Exception(result["status"])
    lat = result["result"]["geometry"]["location"]["lat"]
    lng = result["result"]["geometry"]["location"]["lng"]

    return (lat, lng)


def find_places(radius_meters: str, lat: float, lng: float, api_key: str) -> str:
    endpoint_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params = {
        "location": f"{lat},{lng}",  # "lat,long" format
        "radius": radius_meters,
        "key": api_key,
        "language": "zh",
    }

    res = requests.get(endpoint_url, params=params)
    results = json.loads(res.content)
    if results["status"] != "OK":
        raise Exception(results["status"])
    places = results["results"]
    if len(places) > 0:
        content_strs = [
            f"{p['name']:45}lat:{p['geometry']['location']['lat']},lng:{p['geometry']['location']['lng']}{'':5}|{'':5}types:{','.join(p['types'])}"
            for p in places
        ]
        return "\n".join(content_strs)
    else:
        return ""

In [3]:
from typing import Any, Dict, List, Optional
from pydantic import Extra, Field
from langchain.chains.base import Chain
from langchain import BasePromptTemplate
from langchain.schema.language_model import BaseLanguageModel
from langchain.chains import LLMChain
from langchain.callbacks.manager import (
    AsyncCallbackManagerForChainRun,
    CallbackManagerForChainRun,
)
from langchain.prompts import PromptTemplate
import os


class NearbySearchChain(Chain):
    """
    An example of a custom chain.
    """

    prompt: BasePromptTemplate
    res_prompt: BasePromptTemplate
    """Prompt object to use."""
    llm: BaseLanguageModel
    output_key: str = "text"  #: :meta private:
    API_KEY: str

    res_chain: Optional[LLMChain] = Field(default=None, exclude=True)

    class Config:
        """Configuration for this pydantic object."""

        extra = Extra.forbid
        arbitrary_types_allowed = True

    @property
    def input_keys(self) -> List[str]:
        """Will be whatever keys the prompt expects.

        :meta private:
        """
        return self.prompt.input_variables

    @property
    def output_keys(self) -> List[str]:
        """Will always return text key.

        :meta private:
        """
        return [self.output_key]

    def _call(
        self,
        inputs: Dict[str, Any],
        run_manager: Optional[CallbackManagerForChainRun] = None,
    ) -> Dict[str, str]:
        prompt_value = self.prompt.format_prompt(**inputs)
        if run_manager:
            run_manager.on_text(
                prompt_value.to_string(), color="green", end="\n", verbose=self.verbose
            )
        result = self.llm.generate_prompt(
            prompts=[prompt_value],
            callbacks=run_manager.get_child() if run_manager else None,
        )
        address = result.generations[0][0].text
        if run_manager:
            run_manager.on_text(address, color="yellow", end="\n", verbose=self.verbose)
        placeid = getPlaceId(address=address, api_key=self.API_KEY)
        (lat, lng) = geocode(placeid=placeid, api_key=self.API_KEY)
        places = find_places(radius_meters=5000, lat=lat, lng=lng, api_key=self.API_KEY)
        res_prompt_value = self.res_prompt.format_prompt(
            address=address, lat=lat, lng=lng, nearby=places
        )
        if run_manager:
            run_manager.on_text(
                res_prompt_value.to_string(),
                color="green",
                end="\n",
                verbose=self.verbose,
            )
        result = self.llm.generate_prompt(
            prompts=[res_prompt_value],
            callbacks=run_manager.get_child() if run_manager else None,
        )
        if run_manager:
            run_manager.on_text(
                result.generations[0][0].text,
                color="green",
                end="\n",
                verbose=self.verbose,
            )
        return {self.output_key: res_prompt_value.to_string()}

    async def _acall(
        self,
        inputs: Dict[str, Any],
        run_manager: Optional[AsyncCallbackManagerForChainRun] = None,
    ) -> Dict[str, str]:
        prompt_value = self.prompt.format_prompt(**inputs)
        if run_manager:
            await run_manager.on_text(
                prompt_value.to_string(), color="green", end="\n", verbose=self.verbose
            )
        result = await self.llm.agenerate_prompt(
            prompts=[prompt_value],
            callbacks=run_manager.get_child() if run_manager else None,
        )
        address = result.generations[0][0].text
        if run_manager:
            await run_manager.on_text(
                address, color="yellow", end="\n", verbose=self.verbose
            )
        placeid = getPlaceId(address=address, api_key=self.API_KEY)
        (lat, lng) = geocode(placeid=placeid, api_key=self.API_KEY)
        places = find_places(
            radius_meters=10000, lat=lat, lng=lng, api_key=self.API_KEY
        )
        res_prompt_value = self.res_prompt.format_prompt(
            address=address, lat=lat, lng=lng, nearby=places
        )
        if run_manager:
            await run_manager.on_text(
                res_prompt_value.to_string(),
                color="green",
                end="\n",
                verbose=self.verbose,
            )
        result = await self.llm.agenerate_prompt(
            prompts=[res_prompt_value],
            callbacks=run_manager.get_child() if run_manager else None,
        )
        if run_manager:
            await run_manager.on_text(
                result.generations[0][0].text,
                color="green",
                end="\n",
                verbose=self.verbose,
            )
        return {self.output_key: res_prompt_value.to_string()}

    @property
    def _chain_type(self) -> str:
        return "nearby_search_chain"

    @classmethod
    def from_llm(
        cls,
        llm: BaseLanguageModel,
        **kwargs: Any,
    ) -> Chain:
        template = """说明
---------------------
- 请把下面问题中包含的地址提取出来

问题
---------------------
{question}

上面问题包含的地址
---------------------"""
        prompt = PromptTemplate.from_template(template=template)
        res_template = """中心地址
---------------------
{address}
Location:{lat},{lng}

地址附近
---------------------
{nearby}

要求
---------------------
- 根据“地址附近”的地点数据生成一段文字。
- 要求包含所有附近地点说明
- 要求说明附近地点与中心地址的距离。
- 要求说明附近地点在中心地址的什么方向
- 要求对附近地点做简要介绍
- 并总结附近地点的种类

Answer
---------------------"""
        res_prompt = PromptTemplate.from_template(template=res_template)
        API_KEY = os.getenv("PLACES_API_KEY")
        return cls(
            llm=llm, prompt=prompt, res_prompt=res_prompt, API_KEY=API_KEY, **kwargs
        )

/tmp/ipykernel_22994/2237817877.py:30: PydanticDeprecatedSince20: `pydantic.config.Extra` is deprecated, use literal values instead (e.g. `extra='allow'`). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.3/migration/
  extra = Extra.forbid


In [4]:
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv

load_dotenv(dotenv_path="env")

chat_gpt = ChatOpenAI(temperature=0.9, model="gpt-3.5-turbo-16k", verbose=True)
chain = NearbySearchChain.from_llm(llm=chat_gpt, verbose=True)
address = input("Input your address: ")
await chain.arun(address)

Input your address:  北京市昌平区昌平路25号




> Entering new NearbySearchChain chain...
说明
---------------------
- 请把下面问题中包含的地址提取出来

问题
---------------------
北京市昌平区昌平路25号

上面问题包含的地址
---------------------
北京市昌平区昌平路25号
中心地址
---------------------
北京市昌平区昌平路25号
Location:40.230248,116.229457

地址附近
---------------------
北京市                                          lat:39.904211,lng:116.407395     |     types:locality,political
大宫门                                          lat:40.24834299999999,lng:116.22348     |     types:tourist_attraction,point_of_interest,establishment
槐树巷                                          lat:40.217069,lng:116.218252     |     types:point_of_interest,establishment
京通线                                          lat:40.228539,lng:116.209035     |     types:point_of_interest,establishment
北京拓然家苑诊所                                     lat:40.1994047,lng:116.2371175     |     types:hospital,point_of_interest,health,establishment
京通线                                          lat:40.234856,lng:116.267586     |     type

'中心地址\n---------------------\n北京市昌平区昌平路25号\nLocation:40.230248,116.229457\n\n地址附近\n---------------------\n北京市                                          lat:39.904211,lng:116.407395     |     types:locality,political\n大宫门                                          lat:40.24834299999999,lng:116.22348     |     types:tourist_attraction,point_of_interest,establishment\n槐树巷                                          lat:40.217069,lng:116.218252     |     types:point_of_interest,establishment\n京通线                                          lat:40.228539,lng:116.209035     |     types:point_of_interest,establishment\n北京拓然家苑诊所                                     lat:40.1994047,lng:116.2371175     |     types:hospital,point_of_interest,health,establishment\n京通线                                          lat:40.234856,lng:116.267586     |     types:point_of_interest,store,establishment\n北京市昌平区粮食局                                    lat:40.21282739999999,lng:116.2316766     |     types:point_of_interest,es